In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#seeding for comparing experiment in part 2
import torch
import json
SEED = 2025
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:2')

# SNLI Bert
## Second Tutorial
https://towardsdatascience.com/fine-tuning-pre-trained-transformer-models-for-sentence-entailment-d87caf9ec9db
Check his Github code for complete notebook. I never referred to it. Medium was enough.
BERT in keras-tf: https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b

In [3]:
# define macros
BERT_MODEL = 'bert-base-german-cased'
batch_size = 4
batches_per_epoch = None

save_path_suffix = 'cotraining_baseline_de_en_allshuffle_de_labelset_balanceen2000_perplexity_'

# Prepare data

## load the dataset

In [9]:
import pandas as pd

# custom reader needed to handle quotechars
def read_df_custom(file, perplexity=False):
    if perplexity:
        header = 'doc	unit1_toks	unit2_toks	unit1_txt	unit2_txt	perplexity_score	original_unit1_txt	original_unit2_txt	sentiment_score_1	sentiment_score_2	sentiment_graded_agreement	sentiment_agreement	s1_toks	s2_toks	unit1_sent	unit2_sent	dir	nuc_children	sat_children	genre	u1_discontinuous	u2_discontinuous	u1_issent	u2_issent	u1_length	u2_length	length_ratio	u1_speaker	u2_speaker	same_speaker	u1_func	u1_pos	u1_depdir	u2_func	u2_pos	u2_depdir	doclen	u1_position	u2_position	percent_distance	distance	lex_overlap_words	lex_overlap_length	unit1_case	unit2_case	label'
        extracted_columns = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label', 'distance', 'u1_depdir', 'u2_depdir', 'u2_func', 'u1_position', 'u2_position', 'sat_children', 'nuc_children', 'genre', 'unit1_case', 'unit2_case',
                                'u1_discontinuous', 'u2_discontinuous', 'same_speaker', 'lex_overlap_length', 'u1_func', 'perplexity_score']
    else:
        header = 'doc	unit1_toks	unit2_toks	unit1_txt	unit2_txt	s1_toks	s2_toks	unit1_sent	unit2_sent	dir	nuc_children	sat_children	genre	u1_discontinuous	u2_discontinuous	u1_issent	u2_issent	u1_length	u2_length	length_ratio	u1_speaker	u2_speaker	same_speaker	u1_func	u1_pos	u1_depdir	u2_func	u2_pos	u2_depdir	doclen	u1_position	u2_position	percent_distance	distance	lex_overlap_words	lex_overlap_length	unit1_case	unit2_case	label'
        extracted_columns = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label', 'distance', 'u1_depdir', 'u2_depdir', 'u2_func', 'u1_position', 'u2_position', 'sat_children', 'nuc_children', 'genre', 'unit1_case', 'unit2_case',
                                'u1_discontinuous', 'u2_discontinuous', 'same_speaker', 'lex_overlap_length', 'u1_func']
    header = header.split()
    df = pd.DataFrame(columns=extracted_columns)
    file = open(file, 'r')

    rows = []
    count = 0 
    for line in file:
        line = line[:-1].split('\t')
        count+=1
        if count ==1: continue
        row = {}
        for column in extracted_columns:
            index = header.index(column)
            try:
                row[column] = line[index]
            except:
                print(count, line)
            row[column] = line[index]
        rows.append(row)

    df = pd.concat([df, pd.DataFrame.from_records(rows)])
    if perplexity:
        df['perplexity_score'] = df['perplexity_score'].astype(float)
    return df

train_df_de = read_df_custom('../../processed/translated/perplexity_top/deu.rst.pcc_train_enriched_translated.rels', perplexity=True)
test_df_de = read_df_custom('../../processed/deu.rst.pcc_test_enriched.rels')
val_df_de = read_df_custom('../../processed/deu.rst.pcc_dev_enriched.rels')
train_df_en = read_df_custom('../../processed/translated/perplexity_top/eng.rst.rstdt_train_enriched_translated.rels', perplexity=True)
test_df_en = read_df_custom('../../processed/translated/perplexity_top/eng.rst.rstdt_test_enriched_translated.rels', perplexity=True)
val_df_en = read_df_custom('../../processed/translated/perplexity_top/eng.rst.rstdt_dev_enriched_translated.rels', perplexity=True)

lang='deu'

## Perplexity average per class

In [5]:
train_df_en.groupby('label')['perplexity_score'].mean()

label
attribution             10954.269459
background                645.946950
cause                     969.636241
comparison               1386.145714
condition                 760.808535
contrast                  578.961405
elaboration              2058.360552
enablement                917.842415
evaluation               1204.624760
explanation               665.707008
joint                    2025.612224
manner-means              932.480977
summary                  3501.154059
temporal                 5331.382970
textual-organization     3955.837917
topic-change             1170.457495
topic-comment            1002.787940
Name: perplexity_score, dtype: float64

In [10]:
train_df_de.groupby('label')['perplexity_score'].mean()

label
antithesis         848.745525
background         494.589449
cause              784.900340
circumstance       504.304861
concession         634.819208
condition          958.825681
conjunction       1541.822773
contrast           695.012255
disjunction        514.471251
e-elaboration      821.025898
elaboration        913.735893
evaluation-n       753.698329
evaluation-s       703.826031
evidence           415.832474
interpretation    1103.010157
joint              838.943208
list              1375.893955
means              629.372303
preparation       1952.404203
purpose            690.524908
reason             622.669783
restatement        395.210284
result             628.991563
sequence          2979.259654
solutionhood       637.784074
summary            282.086824
Name: perplexity_score, dtype: float64